<a href="https://colab.research.google.com/github/Kalwerake/Complement-Cross-Entropy/blob/master/CRNN4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNN-LSTM 4**

> layers remain the same. Slight changes in LSTM layer, fixed error in output reshaping. 
> using dfc with time window 22





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=236308 sha256=3c5fddb9433565ce048ae7dabe910fff48ddaa6419b9197904f7b07373cc4230
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


In [ ]:
!pip install pandas=='1.5.2'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import pickle5 as pickle
import sys
from matplotlib import pyplot as plt

import torch
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

In [ ]:
pd.__version__

'1.5.2'

In [ ]:
main_dir = '/content/drive/MyDrive/Project'
data_dir = '/content/drive/MyDrive/Project/dfc_cc200'
pheno_dir = '/content/drive/MyDrive/Project/phenotype_files'

In [ ]:
train_df_p = '/content/drive/MyDrive/Project/phenotype_files/train_nn.csv'
test_df_p = '/content/drive/MyDrive/Project/phenotype_files/test_nn.csv'

In [ ]:
train_df = pd.read_csv(train_df_p)
test_df = pd.read_csv(test_df_p)

In [ ]:
class DfcDataset(Dataset):


    def __init__(self,df,dfc_dir):
        """
        :param df: pandas dataframe containing subject ids in column `FILE_ID` and target labels in column `TARGET`
        :param dfc_dir: path to subdirectory containing the pickle files
        """
        self.df_main = df
        self.dfc_dir = dfc_dir
        self.paths = [os.path.join(dfc_dir,f + '_dfc.pkl') for f in df.FILE_ID] # paths to pkl files

    def __getitem__(self, idx):
        """
        :param idx: index
        :return: 4D tensor with 2 spatial dimensions and 1 temporal [N,1,90,200,200], and label
        """
        dfc_pickle = pickle.load(file=open(self.paths[idx], "rb")) # load pkl file as dictionary

        dfc_array = [val.to_numpy() for val in dfc_pickle.values()] # convert each dict value to a 2d list and make nested list of all values
        dfc_tensor = [torch.from_numpy(npDfc).type(torch.float) for npDfc in dfc_array] #take each list in dfc_array and turn to torch tensor of type torch.float, otherwise result is float(64)

        dfc_stack = torch.stack(dfc_tensor) # make tensor stack of each dfc array

        label = self.df_main.loc[idx, 'TARGET'] # class label of idx as np array
        label_tensor = torch.tensor(label)
        label_tensor = label_tensor.to(torch.float32)




        return torch.unsqueeze(dfc_stack, dim= 0),torch.unsqueeze(label_tensor, dim = -1) # turn torch stack to 4D tensor for input into conv3d (channel,depth,height,width), return label as tensor of shape [batch size, 1]

    def __len__(self):
        """
        :return: length of dataset
        """
        return len(self.df_main)

In [ ]:
class CRNN4(nn.Module):
    def __init__(self):
        super(CRNN4, self).__init__()

        #Convolutional layers
        self.seq_cnn = nn.Sequential(
            nn.Conv3d(in_channels=1,out_channels=8, kernel_size=(2,200,1),stride=(1,1,1)),
            nn.ReLU(),
            nn.Conv3d(in_channels=8,out_channels=16, kernel_size=(2,1,200),stride=(1,1,1)),
            nn.ReLU(),
            nn.Conv3d(in_channels=16,out_channels=32, kernel_size=(8,1,1),stride=(2,1,1)),
            nn.ReLU()
        )



        #lstm layer
        self.lstm1 = nn.LSTM(input_size=32, hidden_size=64, num_layers=1, batch_first = True)

        # Fully connected layer
        self.seq_dense = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(16,1),
            nn.Sigmoid()
        )


    def forward(self, x):

        x = self.seq_cnn(x)


        x = torch.flatten(x,start_dim=2,end_dim=4) # reduce dimensionality for LSTM layer, to 3D tensor
        x = x.permute(0,2,1) # transpose to make tensor of size [batch_size, sequence length, feature number]

        x,_ = self.lstm1(x)

        x = x[:,-1,:] # take output of last LSTM cell

        x = self.seq_dense(x)

        return x

In [ ]:
train_data = DfcDataset(train_df, data_dir)
train_dataloader = DataLoader(train_data, batch_size= 16,
                                        shuffle=True,
                                        num_workers=0)
        
test_data = DfcDataset(test_df, data_dir)
test_dataloader = DataLoader(test_data, batch_size= 16,
                                        shuffle=True,
                                        num_workers=0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model4 = CRNN4()
loss_fn = nn.BCELoss()
optimiser =  optim.Adam(model4.parameters(), lr=0.001)

In [ ]:
model4.to(device)

CRNN4(
  (seq_cnn): Sequential(
    (0): Conv3d(1, 8, kernel_size=(2, 200, 1), stride=(1, 1, 1))
    (1): ReLU()
    (2): Conv3d(8, 16, kernel_size=(2, 1, 200), stride=(1, 1, 1))
    (3): ReLU()
    (4): Conv3d(16, 32, kernel_size=(8, 1, 1), stride=(2, 1, 1))
    (5): ReLU()
  )
  (lstm1): LSTM(32, 64, batch_first=True)
  (seq_dense): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.25, inplace=False)
    (6): Linear(in_features=16, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [ ]:
train_loss_history = []
train_acc_history = []
test_loss_history = []
test_acc_history = []
for i in range(200):
  
  running_loss = 0
  epoch_accuracy = []
  model4.train()
  for j, (x_train,y) in enumerate(train_dataloader):
    optimiser.zero_grad() # zero the gradient at each epoch start
    y = y.to(device) # send y to cuda
    x_train = x_train.to(device)
    prediction = model4.forward(x_train)
    loss = loss_fn(prediction,y) # loss

    accuracy = (torch.round(prediction) == y).float().mean() # calculate accuracy for each mini-batch  take prediction tensor, reshape to 1d detach from computational graph turn to numpy array, round and see if rounded number is equal to label, find mean of this boolean array, this is the accuracy

    running_loss += loss.item() # get epoch loss
    epoch_accuracy.append(accuracy.item())
                

    loss.backward() # backward propgation
    optimiser.step()
  
  mean_acc_train = np.mean(epoch_accuracy)
  train_loss_history.append(running_loss)
  train_acc_history.append(mean_acc_train)
  
  model4.eval()
  test_loss_run = 0
  test_acc_epoch = []
  for j, (x_test,y_test) in enumerate(test_dataloader):
    y_test = y_test.to(device)
    x_test= x_test.to(device)
    test_pred = model4.forward(x_test)
    test_loss = loss_fn(test_pred,y_test) # loss

    test_acc = (torch.round(test_pred) == y_test).float().mean() # calculate accuracy for each mini-batch  take prediction tensor, reshape to 1d detach from computational graph turn to numpy array, round and see if rounded number is equal to label, find mean of this boolean array, this is the accuracy

    test_loss_run += test_loss.item()
    test_acc_epoch.append(test_acc.item())

  mean_acc_test = np.mean(test_acc_epoch)
  test_acc_history.append(mean_acc_test)
  test_loss_history.append(test_loss_run)
  if (i+1)%5 == 0:
    print(f'Epoch {i+1} train_loss: {round(running_loss,2)}, accuracy: {round(mean_acc_train,2)}, test_loss:{round(test_loss_run,2)}, test_acc: {round(mean_acc_test,2)} ')
            



Epoch 5 train_loss: 22.75, accuracy: 0.76, test_loss:8.96, test_acc: 0.59 
Epoch 10 train_loss: 10.01, accuracy: 0.92, test_loss:11.45, test_acc: 0.68 
Epoch 15 train_loss: 4.41, accuracy: 0.97, test_loss:20.4, test_acc: 0.61 
Epoch 20 train_loss: 5.0, accuracy: 0.97, test_loss:25.58, test_acc: 0.59 
Epoch 25 train_loss: 2.24, accuracy: 0.99, test_loss:21.69, test_acc: 0.61 
Epoch 30 train_loss: 1.97, accuracy: 0.99, test_loss:24.95, test_acc: 0.61 
Epoch 35 train_loss: 1.91, accuracy: 0.99, test_loss:25.73, test_acc: 0.63 
Epoch 40 train_loss: 2.77, accuracy: 0.98, test_loss:21.18, test_acc: 0.64 
Epoch 45 train_loss: 4.57, accuracy: 0.97, test_loss:19.49, test_acc: 0.6 
Epoch 50 train_loss: 1.64, accuracy: 0.99, test_loss:24.91, test_acc: 0.64 
Epoch 55 train_loss: 1.01, accuracy: 0.99, test_loss:28.21, test_acc: 0.63 
Epoch 60 train_loss: 1.39, accuracy: 0.99, test_loss:31.67, test_acc: 0.63 
Epoch 65 train_loss: 0.81, accuracy: 1.0, test_loss:36.6, test_acc: 0.62 
Epoch 70 train_lo

KeyboardInterrupt: ignored

In [ ]:
torch.save(model4.state_dict(), '/content/drive/MyDrive/Project/model_params/CRNN4_params.pth') #save model parameters in google drive as pickle

In [ ]:

crnn2_train_metrics = {'train_acc_history': acc_history, 'test_loss_history': loss_history, 'test_acc': test_acc_history, 'test_loss': test_loss_history} # make dictionary of metrics

mod_eval_p = '/content/drive/MyDrive/Project/model_evaluation'# path to subdirectory for model evaluation storage

met_path = os.path.join(mod_eval_p, 'crnn4_train_metrics.pickle') # path to pickle name for metric storage

with open(met_path, 'wb') as handle:
    pickle.dump(crnn2_train_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: ignored